In [ ]:
import ipytest
import pytest

ipytest.autoconfig()

In [ ]:
%%ipytest -s

@pytest.fixture(scope="module")
def module_scope_fixture():
    print("\nSetup module fixture")
    yield
    print("\nTeardown module fixture")
    
def test_one(module_scope_fixture):
    assert True
    
def test_two(module_scope_fixture):
    assert True


In [ ]:
%%ipytest -s

@pytest.fixture(params=[0, 1, 2])
def num(request):
    print(type(request))
    return request.param

def test_numbers(num):
    print(num)
    assert num in [0, 1, 2]


In [ ]:
%%ipytest -s


def create_client():
    class Client:
        def login(self, *args, **kwargs):
            self._is_authenticated = True
            print("client logged in")
        
        @property
        def is_authenticated(self):
            return hasattr(self, '_is_authenticated')
    
    return Client()


@pytest.fixture
def user_data():
    return {"username": "testuser", "password": "password"}


@pytest.fixture
def authenticated_client(user_data):
    client = create_client()
    client.login(user_data["username"], user_data["password"])

    return client


def test_authenticated(authenticated_client):
    assert authenticated_client.is_authenticated


In [ ]:
%%ipytest -s

# Stub objects replace real objects when you only care about returning a specific result but do not care about tracking interactions with the object.

class PaymentService:
    def charge(self, amount) -> bool:
        pass # complicated logic is here


class Order:
    def __init__(self, amount: int):
        self.amount = amount
    
def process_order(order, payment_service):
    payment_service.charge(order.amount)


# stub
class PaymentServiceStub:
    def charge(self, amount):
        return True
    
    
def test_order_payment():
    order = Order(amount=100)
    payment_service_stub = PaymentServiceStub()
    process_order(order, payment_service_stub)


In [ ]:
%%ipytest -s
# Fake objects are used when you need a working implementation that is simpler and faster than the real thing.

class InMemoryDatabase:
    def __init__(self):
        self.data = {}

    def save(self, record):
        self.data[record.id] = record

    def find(self, record_id):
        return self.data.get(record_id)
    

class Record:
    def __init__(self, id: int, name: str):
        self.id = id
        self.name = name
    
    
def test_database_save():
    db = InMemoryDatabase()
    record = Record(id=1, name='Test')
    db.save(record)
    saved_record = db.find(1)
    assert saved_record.name == 'Test'


In [ ]:
%%ipytest -s

from abc import ABC, abstractmethod


# Mocks are used to verify interactions, ensuring that methods were called with the correct parameters, at the right time, and the correct number of times.

from unittest.mock import Mock


class PaymentProcessorInterface(ABC):
    @abstractmethod
    def pay(self, amount):
        pass


class GooglePayProcessor(PaymentProcessorInterface):
    def pay(self, amount):
        # complicated logic
        pass


class ApplePayProcessor(PaymentProcessorInterface):
    def pay(self, amount):
        # complicated logic
        pass


class PaymentService:
    def __init__(self, processor: PaymentProcessorInterface):
        self.processor = processor

    def process_payment(self, amount):
        return self.processor.pay(amount)
    
    
def test_payment():
    mock_processor = Mock(spec=PaymentProcessorInterface)
    # mock_processor = Mock()
    # mock_processor.foo.return_value = True
    mock_processor.pay.return_value = True

    service = PaymentService(mock_processor)
    assert service.process_payment(100) is True
    mock_processor.pay.assert_called_once_with(100)

In [ ]:
%%ipytest -s

def test_create_and_verify_temp_file(tmp_path):
    # Create a temporary directory within tmp_path
    temporary_directory = tmp_path / "example_temp_dir"
    temporary_directory.mkdir()

    # Create a file inside the temporary directory
    temporary_file = temporary_directory / "example_file.txt"
    temporary_file.write_text("Temporary file content")

    # Verify that the file exists
    assert temporary_file.is_file()

    # Verify the file's contents
    assert temporary_file.read_text() == "Temporary file content"